# Assign addresses to the 3D BAG models

In this tutorial we query the BAG API to get the address information for each building model, and assign the addresses to the models.

For this we work the [BAG APIv2](https://www.kadaster.nl/zakelijk/producten/adressen-en-gebouwen/bag-api-individuele-bevragingen).
In order to access the BAG API, you need to [request an API Key](https://formulieren.kadaster.nl/aanvraag_bag_api_individuele_bevragingen_productie).
The [BAG API is documentation](https://lvbag.github.io/BAG-API/Technische%20specificatie/) gives the details on the available endpoints.
More information can be found in the [GitHub repo of the API](https://github.com/lvbag/BAG-API).
The [official BAG data catalogue](https://www.geobasisregistraties.nl/documenten/publicatie/2018/03/12/catalogus-2018) has all the details about the BAG.

In [1]:
import os
from pathlib import Path
import json
import requests
from cjio import cityjson

In [2]:
with Path(".env").resolve().open("r") as fo:
    env_vars = fo.readlines()
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

BAG_API = "https://api.bag.acceptatie.kadaster.nl/lvbag/individuelebevragingen/v2"
headers = {"Accept": "application/hal+json",
           "X-Api-Key": os.environ.get("BAG_API_KEY")}

Open a session with the API key in the header, so that we can use it for the rest of the queries.

In [3]:
s = requests.Session()
s.headers.update(headers)

Query a municipality just to test that the API works.

In [4]:
municipality_id = "0518"
res = s.get(f"{BAG_API}/bronhouders/{municipality_id}").json()
print(res["bronhouder"]["code"], res["bronhouder"]["naam"], res["bronhouder"]["provincie"]["naam"])

0518 's-Gravenhage Zuid-Holland


We load a citymodel from the 3D BAG, and isolate one building.

In [5]:
cm_path = (Path("data") / "sample.json").resolve()
cm = cityjson.load(cm_path, transform=True)
cos = cm.get_cityobjects(id="NL.IMBAG.Pand.0518100000285075")
one = cos["NL.IMBAG.Pand.0518100000285075"]

one_children = cm.get_cityobjects(id=one.children)
print(f"The CityObject {one.id} of type {one.type} has {len(one.children)} children.\n"
      f"They are: {one.children}, of type {list(one_children.values())[0].type}")
one_bpart = one_children["NL.IMBAG.Pand.0518100000285075-0"]

The CityObject NL.IMBAG.Pand.0518100000285075 of type Building has 1 children.
They are: ['NL.IMBAG.Pand.0518100000285075-0'], of type BuildingPart


Get the addresses within a CityObject. For this, we need to parse the CityObject ID into a BAG Pand ID, so that we can query the API with the Pand ID.

The CityObject ID is built up like this:

`namespace` . `object type` . `object ID` - `object part ID`

Where `namespace` is always `NL.IMBAG`, `object type` is always `Pand`, the `object ID` is the BAG Pand ID, the `object part ID` is a sequential ID for building parts.

In [6]:
co_id = one.id
pand_id = co_id.replace("NL.IMBAG.Pand.", "").rsplit("-")[0]

There can be multiple addresses within one building, so we collect them and assign
them to the BuildingPart.

In [7]:
addresses = {}
res = s.get(f"{BAG_API}/adressen", params={"pandIdentificatie": pand_id}).json()
for i, a in enumerate(res["_embedded"]["adressen"]):
    print(f'{a["openbareRuimteNaam"]} {a["huisnummer"]}, {a["postcode"]} {a["woonplaatsNaam"]}')
    addresses[i] = {
        "openbareRuimteNaam": a["openbareRuimteNaam"],
        "huisnummer": a["huisnummer"],
        "postcode": a["postcode"],
        "woonplaatsNaam": a["woonplaatsNaam"]
    }

C.A. van Beverenplein 3, 2552HT 's-Gravenhage
C.A. van Beverenplein 4, 2552HT 's-Gravenhage
C.A. van Beverenplein 5, 2552HT 's-Gravenhage
C.A. van Beverenplein 6, 2552HT 's-Gravenhage
C.A. van Beverenplein 7, 2552HT 's-Gravenhage


In the 3D BAG the parent Building objects represent the BAG objects and the Building-s
store all the attributes that are true for the sholw BAG object. Therefore, we also put
the addresses on the Building.

---

**Note:**

The address definition for CityObjects as we did here is strictly speaking not valid
according to the CityJSON specificaton. CityJSON only allows a single address per
Building(Part), and we used the fields (huisnummer, postcode etc.) from the BAG,
instead the fields prescribed by the [xAL address standard](http://xml.coverpages.org/xnal.html). See the [CityJSON specs for details](https://www.cityjson.org/specs/1.0.3/#building).

However, it is likely that the address definition will change in CityJSON
(see [this issue](https://github.com/cityjson/specs/issues/51)), and using the Dutch
standard address definition for a Dutch data set like the 3D BAG is sensible.

---

In [8]:
one.address = addresses

If you stay in python and don't need to write out the CityObjects with addresses to a
CityJSON file, then that's it, we are done.

If you need to write a CityJSON file with the addresses, the addresses need to be
added manually to the json. This is, because the cjio API currently doesn't support
properties that are specific to a certain CityObject type. Such as the `address` is
specific to the `Building` and `BuildingPart` CityObject types.

This means that we won't use the `cityjson.save()` method, but need to do the steps
by hand.

First convert the geometry and other data back to the schema that will be written to
the json file. Also take care to remove the duplicate and orphan vertices from the
geometries, so that we can get a leaner file.

In [9]:
# Note the 'j' in the method's name add_to_j()
cm.add_to_j()

cm.remove_duplicate_vertices()
cm.remove_orphan_vertices()

0

Then add the addresses to the objects which contain them. Since we dereferenced the
geometries and used the method `add_to_j`, all the cityobject data is stored with the
CityJSON schema on the `j` attribute of the citymodel. Thus, we need to add the
addresses directly to the schema in the `j` attribute.

In [10]:
for co_id, co in cm.cityobjects.items():
    if hasattr(co, "address"):
        cm.j["CityObjects"][co_id]["address"] = co.address

Write the contents from `j` to file, which will give us a CityJSON file.

In [11]:
with Path("outfile.json").open("w") as fout:
    json_str = json.dumps(cm.j, separators=(',',':'))
    fout.write(json_str)

Close the session when done.

In [12]:
s.close()

## Explore further
In this tutorial we assigned the addresses to the building models based on their ID.
However, some large buildings are split up into multiple parts in the 3D BAG.
Each part has the same Pand ID in the BAG, because they are modelled with a single polygon.
Thus, these parts get all the same addresses from the Pand.
It would be better to match the address points to the corresponding building parts.